In [1]:
import os
import pandas as pd
import numpy as np
from datetime import date, timedelta
import matplotlib.pyplot as plt

In [2]:
def calcTaxPA(grossSalaryPA):
    '''calculates tax paid per year based on 
    UK tax bands'''
    taxFreeAllowance = 12500
    if grossSalaryPA <= taxFreeAllowance:
        taxPA = 0
    elif grossSalaryPA <= 50000:
        taxPA = 0.2*(grossSalaryPA - 12500)
    elif grossSalaryPA > 50000 and grossSalaryPA <= 150000:
        taxPA = (0.4*(grossSalaryPA - 50000)) + (0.2*(50000 - 12500))
    elif grossSalaryPA > 150000:
        taxPA = (0.45*(grossSalaryPA - 150000)) + (0.4*(150000 - 50000)) + (0.2*(50000 - 12500))
    return taxPA


def calcNICPA(grossSalaryPA):
    '''calculates national insurance contribution per year
    based on UK bands'''
    grossSalaryPW = grossSalaryPA/52
    if grossSalaryPW >= 962:
        NICPW = 0.02*(grossSalaryPW-962) + 0.12*(962-166)
    elif grossSalaryPW > 166:
        NICPW = 0.12*(grossSalaryPW - 166)
    else:
        NICPW = 0
    return NICPW*52


def calcStudentFinPA(grossSalaryPA):
    '''calculates student finance per year'''
    grossSalaryPM = grossSalaryPA/12
    thresholdPM = 2143
    if grossSalaryPM > thresholdPM:
        StudentFinPM = 0.09*(grossSalaryPM-thresholdPM)
    else:
        StudentFinPM = 0
    return StudentFinPM*12


def calcNetSalary(grossSalaryPA):
    '''calculates net salary after tax and all other deductions'''
    netSalaryPA = grossSalaryPA - calcTaxPA(grossSalaryPA) - calcNICPA(grossSalaryPA) - calcStudentFinPA(grossSalaryPA)
    return netSalaryPA


def salaryIncrease(grossSalaryPA, mean, sd):
    '''increase salary my random amount drawn from gaussian'''
    increaseStep = np.random.normal(mean, sd)
    return grossSalaryPA+increaseStep

def findInterestRate(paymentDate):
    year = paymentDate.year
    sdate = historicInterest[str(year)]["start"]
    edate = historicInterest[str(year)]["end"]
    delta = edate - sdate
    
    allDates = [sdate + timedelta(days=i) for i in range(delta.days+1)]

    if paymentDate in allDates:
        rate = historicInterest[str(year)]["rate"]
    else:
        rate = historicInterest[str(year-1)]["rate"]
    return rate

def graduation_amount(myPayments):
    for i, row in myPayments.iterrows():
    #     print(row)
        rate = findInterestRate(myPayments.at[i, "PaymentDate"].date())
    
def populateInterestRate(myPayments):
    myPayments["interestRate"] = 0
    myPayments["totalPayment"] = 0
    myPayments["cumulativeTotal"] = 0
    for i, row in myPayments.iterrows():
    #     print(row)
        rate = findInterestRate(myPayments.at[i, "PaymentDate"].date())
#         print(rate)
        myPayments.loc[i,"interestRate"] = rate
#         print(rate+1)
        myPayments.loc[i, "totalPayment"] = myPayments.loc[i, "Gross"]*(rate+1)
        myPayments.loc[i, "cumulativeTotal"] = myPayments.loc[i, "Gross"]*(rate+1)
    return myPayments

In [3]:
## build this dictionary from a table which is read in as a csv - potentially even scrape from the student finance website

historicInterest = {"2012": {"start": date(2012, 9, 1), "end": date(2013, 8, 31), "rate": 0.066}, 
                    "2013": {"start": date(2013, 9, 1), "end": date(2014, 8, 31), "rate": 0.063}, 
                    "2014": {"start": date(2014, 9, 1), "end": date(2015, 8, 31), "rate": 0.055}, 
                    "2015": {"start": date(2015, 9, 1), "end": date(2016, 8, 31), "rate": 0.039}, 
                    "2016": {"start": date(2016, 9, 1), "end": date(2017, 8, 31), "rate": 0.046}, 
                    "2017": {"start": date(2017, 9, 1), "end": date(2018, 8, 31), "rate": 0.0618}, 
                    "2018": {"start": date(2018, 9, 1), "end": date(2019, 8, 31), "rate": 0.063}
                   }


# incomeTax = {12500: 0.0, 50000: 0.2, 150000: 0.4, 0.45}
current_wd = os.getcwd()
file_name = "trimmed_data.csv"

def readPayments(file_name, date_header="PaymentDate"):
    dataLoc = os.path.join(current_wd, "data", file_name)
    paymentsData = pd.read_csv(dataLoc)
    paymentsData[date_header] = pd.to_datetime(paymentsData[date_header])
    
    return paymentsData

In [4]:
## Read in data

myPayments = pd.read_csv("trimmed_data.csv", sep=",")
myPayments["PaymentDate"] = pd.to_datetime(myPayments["PaymentDate"])

In [5]:
myPayments

,PaymentDate,Instalment,Gross
0,2014-09-25,1,1191.30
1,2014-10-15,1,2250.00
2,2015-01-05,2,1191.30
3,2015-02-04,2,2250.00
4,2015-04-20,3,1227.40
5,2015-05-06,3,4500.00
6,2015-09-24,4,1231.23
7,2015-10-21,4,2250.00
8,2016-01-04,5,1231.23
9,2016-02-03,5,2250.00


In [6]:
myPayments = populateInterestRate(myPayments)
myPayments

,PaymentDate,Instalment,Gross,interestRate,totalPayment,cumulativeTotal
0,2014-09-25,1,1191.30,0.055,1256.82150,1256.82150
1,2014-10-15,1,2250.00,0.055,2373.75000,2373.75000
2,2015-01-05,2,1191.30,0.055,1256.82150,1256.82150
3,2015-02-04,2,2250.00,0.055,2373.75000,2373.75000
4,2015-04-20,3,1227.40,0.055,1294.90700,1294.90700
5,2015-05-06,3,4500.00,0.055,4747.50000,4747.50000
6,2015-09-24,4,1231.23,0.039,1279.24797,1279.24797
7,2015-10-21,4,2250.00,0.039,2337.75000,2337.75000
8,2016-01-04,5,1231.23,0.039,1279.24797,1279.24797
9,2016-02-03,5,2250.00,0.039,2337.75000,2337.75000


In [7]:
def simulateLifetimeEarnings(grossSalaryPA, N = 40):
    lifetime_tax = 0
    lifetime_SF = 0
    lifetime_NIC = 0
    lifetime_earning = 0
    
    for i in range(N):
#         print(i)
        if i%5 == 0 and i > 0 and i < 20:
            grossSalaryPA = salaryIncrease(grossSalaryPA, mean=5000, sd=3000)
        tax = calcTaxPA(grossSalaryPA)
        lifetime_tax += tax
        NIC = calcNICPA(grossSalaryPA)
        lifetime_NIC += NIC
        SF = calcStudentFinPA(grossSalaryPA)
        lifetime_SF += SF
        lifetime_earning += grossSalaryPA
        print("salary: ", grossSalaryPA, ". Student finance cumulative: ", lifetime_SF)
        if i == 30:
            print("***************30 YEARS***************")
        
    print("Across {} years of earning \nat a final gross salary of {} per year \nYou paid a total of: \n{} in tax \n{} in NIC \n{} in Student Finance".format(N, grossSalaryPA, lifetime_tax, lifetime_NIC, lifetime_SF))
    print("Across {} years of earning \nat you earned a cumulative gross salary of {}".format(N, lifetime_earning))

In [8]:
print(payEnd)

NameError: name 'payEnd' is not defined

In [9]:
def graduateAmount(myPayments=None, simEnd="yearEnd"):
    cumulativeTotal = 0
    interestRate = 0
    if myPayments is not None:
        startDate = min(myPayments.PaymentDate)
        
        if simEnd == "yearEnd":
            graduationYear = max(myPayments.PaymentDate).year
            yearEnd = str(graduationYear)+"-08-31"
            endDate = pd.Timestamp(yearEnd) # simulation ends at end of academic year of final payment
        else:
            endDate = max(myPayments.PaymentDate) # simulation ends at final payment
            
        paymentDates = list(myPayments.PaymentDate)
        delta = timedelta(days=1)
        
        while startDate <= endDate: # up to and including final payment
            interestRate = findInterestRate(startDate)
            cumulativeTotal *= (1+(interestRate/365)) # apply interest on previous payments before new payment
            if startDate in paymentDates:
#                 interestRate = myPayments.loc[myPayments.PaymentDate == startDate, "interestRate"].values[0]
                cumulativeTotal += myPayments.loc[myPayments.PaymentDate == startDate, "Gross"].values[0]
            print(startDate, interestRate)
            startDate += delta                
            
    else:
        print("Please enter your net total of Student Debt at graduation")
        cumulativeTotal = input()
    return cumulativeTotal

graduateAmount = graduateAmount(myPayments)

2014-09-25 00:00:00 0.055
2014-09-26 00:00:00 0.055
2014-09-27 00:00:00 0.055
2014-09-28 00:00:00 0.055
2014-09-29 00:00:00 0.055
2014-09-30 00:00:00 0.055
2014-10-01 00:00:00 0.055
2014-10-02 00:00:00 0.055
2014-10-03 00:00:00 0.055
2014-10-04 00:00:00 0.055
2014-10-05 00:00:00 0.055
2014-10-06 00:00:00 0.055
2014-10-07 00:00:00 0.055
2014-10-08 00:00:00 0.055
2014-10-09 00:00:00 0.055
2014-10-10 00:00:00 0.055
2014-10-11 00:00:00 0.055
2014-10-12 00:00:00 0.055
2014-10-13 00:00:00 0.055
2014-10-14 00:00:00 0.055
2014-10-15 00:00:00 0.055
2014-10-16 00:00:00 0.055
2014-10-17 00:00:00 0.055
2014-10-18 00:00:00 0.055
2014-10-19 00:00:00 0.055
2014-10-20 00:00:00 0.055
2014-10-21 00:00:00 0.055
2014-10-22 00:00:00 0.055
2014-10-23 00:00:00 0.055
2014-10-24 00:00:00 0.055
2014-10-25 00:00:00 0.055
2014-10-26 00:00:00 0.055
2014-10-27 00:00:00 0.055
2014-10-28 00:00:00 0.055
2014-10-29 00:00:00 0.055
2014-10-30 00:00:00 0.055
2014-10-31 00:00:00 0.055
2014-11-01 00:00:00 0.055
2014-11-02 0

2016-01-06 00:00:00 0.039
2016-01-07 00:00:00 0.039
2016-01-08 00:00:00 0.039
2016-01-09 00:00:00 0.039
2016-01-10 00:00:00 0.039
2016-01-11 00:00:00 0.039
2016-01-12 00:00:00 0.039
2016-01-13 00:00:00 0.039
2016-01-14 00:00:00 0.039
2016-01-15 00:00:00 0.039
2016-01-16 00:00:00 0.039
2016-01-17 00:00:00 0.039
2016-01-18 00:00:00 0.039
2016-01-19 00:00:00 0.039
2016-01-20 00:00:00 0.039
2016-01-21 00:00:00 0.039
2016-01-22 00:00:00 0.039
2016-01-23 00:00:00 0.039
2016-01-24 00:00:00 0.039
2016-01-25 00:00:00 0.039
2016-01-26 00:00:00 0.039
2016-01-27 00:00:00 0.039
2016-01-28 00:00:00 0.039
2016-01-29 00:00:00 0.039
2016-01-30 00:00:00 0.039
2016-01-31 00:00:00 0.039
2016-02-01 00:00:00 0.039
2016-02-02 00:00:00 0.039
2016-02-03 00:00:00 0.039
2016-02-04 00:00:00 0.039
2016-02-05 00:00:00 0.039
2016-02-06 00:00:00 0.039
2016-02-07 00:00:00 0.039
2016-02-08 00:00:00 0.039
2016-02-09 00:00:00 0.039
2016-02-10 00:00:00 0.039
2016-02-11 00:00:00 0.039
2016-02-12 00:00:00 0.039
2016-02-13 0

2017-05-20 00:00:00 0.046
2017-05-21 00:00:00 0.046
2017-05-22 00:00:00 0.046
2017-05-23 00:00:00 0.046
2017-05-24 00:00:00 0.046
2017-05-25 00:00:00 0.046
2017-05-26 00:00:00 0.046
2017-05-27 00:00:00 0.046
2017-05-28 00:00:00 0.046
2017-05-29 00:00:00 0.046
2017-05-30 00:00:00 0.046
2017-05-31 00:00:00 0.046
2017-06-01 00:00:00 0.046
2017-06-02 00:00:00 0.046
2017-06-03 00:00:00 0.046
2017-06-04 00:00:00 0.046
2017-06-05 00:00:00 0.046
2017-06-06 00:00:00 0.046
2017-06-07 00:00:00 0.046
2017-06-08 00:00:00 0.046
2017-06-09 00:00:00 0.046
2017-06-10 00:00:00 0.046
2017-06-11 00:00:00 0.046
2017-06-12 00:00:00 0.046
2017-06-13 00:00:00 0.046
2017-06-14 00:00:00 0.046
2017-06-15 00:00:00 0.046
2017-06-16 00:00:00 0.046
2017-06-17 00:00:00 0.046
2017-06-18 00:00:00 0.046
2017-06-19 00:00:00 0.046
2017-06-20 00:00:00 0.046
2017-06-21 00:00:00 0.046
2017-06-22 00:00:00 0.046
2017-06-23 00:00:00 0.046
2017-06-24 00:00:00 0.046
2017-06-25 00:00:00 0.046
2017-06-26 00:00:00 0.046
2017-06-27 0

In [10]:
print(graduateAmount)

40663.4864973795


In [250]:
sdate = historicInterest["2017"]["start"]
edate = date(2018,3,31)
interestRate = historicInterest["2017"]["rate"]
interestRate = 0.061
print(interestRate)
cumulativeTotal = graduateAmount
while sdate <= edate:
    print(sdate)
    cumulativeTotal *= (1+(interestRate/365))
    sdate += delta  
cumulativeTotal

0.061
2017-09-01
2017-09-02
2017-09-03
2017-09-04
2017-09-05
2017-09-06
2017-09-07
2017-09-08
2017-09-09
2017-09-10
2017-09-11
2017-09-12
2017-09-13
2017-09-14
2017-09-15
2017-09-16
2017-09-17
2017-09-18
2017-09-19
2017-09-20
2017-09-21
2017-09-22
2017-09-23
2017-09-24
2017-09-25
2017-09-26
2017-09-27
2017-09-28
2017-09-29
2017-09-30
2017-10-01
2017-10-02
2017-10-03
2017-10-04
2017-10-05
2017-10-06
2017-10-07
2017-10-08
2017-10-09
2017-10-10
2017-10-11
2017-10-12
2017-10-13
2017-10-14
2017-10-15
2017-10-16
2017-10-17
2017-10-18
2017-10-19
2017-10-20
2017-10-21
2017-10-22
2017-10-23
2017-10-24
2017-10-25
2017-10-26
2017-10-27
2017-10-28
2017-10-29
2017-10-30
2017-10-31
2017-11-01
2017-11-02
2017-11-03
2017-11-04
2017-11-05
2017-11-06
2017-11-07
2017-11-08
2017-11-09
2017-11-10
2017-11-11
2017-11-12
2017-11-13
2017-11-14
2017-11-15
2017-11-16
2017-11-17
2017-11-18
2017-11-19
2017-11-20
2017-11-21
2017-11-22
2017-11-23
2017-11-24
2017-11-25
2017-11-26
2017-11-27
2017-11-28
2017-11-29
2017

42129.901114597546

# To-do
* Function to calculate student finance owed - need this as separate so can calculate cumulative total over simulated years 

In [8]:
calcTaxPA(35000-(0.05*35000))/12
calcNICPA(35000-(0.05*35000))/12
calcStudentFinPA(35000-(0.05*35000))/12

56.50500000000002